## **LAB 2**

## _TASK 1_

CodeCarbon library and necessary imports for the lab.

**For MacOS**

In [25]:
%pip install -q tensorflow codecarbon

Note: you may need to restart the kernel to use updated packages.


In [26]:
%pip install -q tensorflow-macos tensorflow-metal codecarbon

ERROR: Could not find a version that satisfies the requirement tensorflow-macos (from versions: none)
ERROR: No matching distribution found for tensorflow-macos
Note: you may need to restart the kernel to use updated packages.


**For COLAB**

In [27]:
%pip install -q codecarbon

Note: you may need to restart the kernel to use updated packages.


In [28]:
!pip install codecarbon

In [ ]:
import os
import random
import time
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from codecarbon import EmissionsTracker
from sklearn.metrics import classification_report, confusion_matrix

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("Physical devices:", tf.config.list_physical_devices())

possible_paths = [
    Path("/content/data"),
    Path("/content/drive/MyDrive/data"),
    Path.cwd() / "data",
    Path.cwd().parent / "data",
]

data_dir = None
for p in possible_paths:
    if p.exists():
        data_dir = p
        break

if data_dir is None:
    raise FileNotFoundError("No se encuentra la carpeta 'data' en ninguna de las rutas posibles")

images_dir = data_dir / "images_original"
genres_dir = data_dir / "genres_original"

print("USING DATA DIR:", data_dir)
print("Images dir exists:", images_dir.exists(), images_dir)
print("Genres dir exists:", genres_dir.exists(), genres_dir)

TensorFlow version: 2.20.0
Physical devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
USING DATA DIR: /Users/meco/Library/CloudStorage/OneDrive-Personal/Educación/UPF/Cursos/Machine Learning for Music/Labs/data
Images dir exists: True /Users/meco/Library/CloudStorage/OneDrive-Personal/Educación/UPF/Cursos/Machine Learning for Music/Labs/data/images_original
Genres dir exists: True /Users/meco/Library/CloudStorage/OneDrive-Personal/Educación/UPF/Cursos/Machine Learning for Music/Labs/data/genres_original


**Construction of the spectograms dataset**

* Dataframe with 2 columns made from images_original.
* Split of 80-20 for training and test. Also within training set, a 20% for validation.

In [30]:
image_filepaths = []
image_labels = []

for genre_name in sorted(os.listdir(images_dir)):
    genre_path = images_dir / genre_name
    if not genre_path.is_dir():
        continue
    for fname in os.listdir(genre_path):
        if fname.lower().endswith((".png", ".jpg", ".jpeg")):
            image_filepaths.append(str(genre_path / fname))
            image_labels.append(genre_name)

df = pd.DataFrame({"filepath": image_filepaths, "label": image_labels})
df = df.sample(frac=1.0, random_state=42).reset_index(drop=True)
print(df.head())
print("Total images:", len(df))
print(df["label"].value_counts())

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

train_df, val_df = train_test_split(
    train_df,
    test_size=0.2,
    stratify=train_df["label"],
    random_state=42
)

print("Train size:", len(train_df))
print("Val size:", len(val_df))
print("Test size:", len(test_df))

                                            filepath    label
0  /Users/meco/Library/CloudStorage/OneDrive-Pers...   hiphop
1  /Users/meco/Library/CloudStorage/OneDrive-Pers...      pop
2  /Users/meco/Library/CloudStorage/OneDrive-Pers...  country
3  /Users/meco/Library/CloudStorage/OneDrive-Pers...    disco
4  /Users/meco/Library/CloudStorage/OneDrive-Pers...      pop
Total images: 999
label
hiphop       100
pop          100
country      100
disco        100
reggae       100
metal        100
rock         100
blues        100
classical    100
jazz          99
Name: count, dtype: int64
Train size: 639
Val size: 160
Test size: 200


**Image generator for CNN**

* Usage of ImageDataGenerator from Keras to load images and rescale them.
* Use of images 128x128, RGB and a batch of 32.
* Use of internal split for normalization and batching.

In [31]:
img_size_cnn = (128, 128)
batch_size = 32

train_datagen_cnn = ImageDataGenerator(rescale=1.0 / 255.0)
val_datagen_cnn = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen_cnn = ImageDataGenerator(rescale=1.0 / 255.0)

train_gen_cnn = train_datagen_cnn.flow_from_dataframe(
    train_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size_cnn,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True
)

val_gen_cnn = val_datagen_cnn.flow_from_dataframe(
    val_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size_cnn,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True
)

test_gen_cnn = test_datagen_cnn.flow_from_dataframe(
    test_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size_cnn,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=False
)

class_indices = train_gen_cnn.class_indices
num_classes = len(class_indices)
print("Classes:", class_indices)
print("Num classes:", num_classes)

Found 639 validated image filenames belonging to 10 classes.
Found 160 validated image filenames belonging to 10 classes.
Found 200 validated image filenames belonging to 10 classes.
Classes: {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}
Num classes: 10


**CNN model trained from 0**

* Compact CNN architecture based from convolutional blocks with Conv2D, MaxPooling2D and Dropout.
* Model ends with a dense layer with Softmax activation equally big as the amount of genres.
* Use of categorical crossentropy as loss functión and Adam optimizer.

In [32]:
def build_cnn_scratch(input_shape, num_classes, base_filters=32, n_blocks=3, dense_units=128, dropout_rate=0.4, learning_rate=1e-3):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    filters = base_filters
    for _ in range(n_blocks):
        model.add(layers.Conv2D(filters, (3, 3), activation="relu", padding="same"))
        model.add(layers.Conv2D(filters, (3, 3), activation="relu", padding="same"))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Dropout(dropout_rate))
        filters *= 2
    model.add(layers.Flatten())
    model.add(layers.Dense(dense_units, activation="relu"))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(num_classes, activation="softmax"))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

input_shape_cnn = (img_size_cnn[0], img_size_cnn[1], 3)
cnn_model = build_cnn_scratch(input_shape_cnn, num_classes)
cnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,482,730 (17.10 MB)

 Trainable params: 4,482,730 (17.10 MB)

 Non-trainable params: 0 (0.00 B)

**CNN Training from 0 with computational effort measured**

* Epochs are set
* Use of model.fit with EmissionsTracker by CodeCarbon. Also using time.time. We get:
    * Average CO2 emissions
    * Energy consumed
    * Experiment duration

In [33]:
epochs_cnn = 20

tracker_cnn = EmissionsTracker(project_name="lab2_cnn_scratch")
tracker_cnn.start()
start_time_cnn = time.time()

history_cnn = cnn_model.fit(
    train_gen_cnn,
    epochs=epochs_cnn,
    validation_data=val_gen_cnn
)

end_time_cnn = time.time()
emissions_cnn = tracker_cnn.stop()

elapsed_cnn = end_time_cnn - start_time_cnn
print("Elapsed time (s):", elapsed_cnn)
print("Estimated CO2 emissions (kg):", emissions_cnn)

[codecarbon WARNING @ 12:14:58] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 12:14:58] Error while trying to count physical CPUs: [Errno 2] No such file or directory: 'lscpu'. Defaulting to 1.
[codecarbon INFO @ 12:14:58] [setup] RAM Tracking...
[codecarbon INFO @ 12:14:58] [setup] CPU Tracking...
[codecarbon WARNING @ 12:14:58] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS and ARM processor detected: Please enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 12:14:58] CPU Model on constant consumption mode: Apple M1 Pro
[codecarbon WARNING @ 12:14:58] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 12:14:58] [setup] GPU Tracking...
[codecarbon INFO @ 12:14:58] No GPU found.
[codecarbon INFO @ 12:14:58] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global const

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 425ms/step - accuracy: 0.0986 - loss: 2.3158 - val_accuracy: 0.1000 - val_loss: 2.3025
Epoch 2/20
13/20 ━━━━━━━━━━━━━━━━━━━━ 2s 365ms/step - accuracy: 0.0961 - loss: 2.3022

[codecarbon INFO @ 12:15:16] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:15:16] Delta energy consumed for CPU with constant : 0.000021 kWh, power : 5.0 W
[codecarbon INFO @ 12:15:16] Energy consumed for All CPU : 0.000021 kWh
[codecarbon INFO @ 12:15:16] 0.000046 kWh of electricity and 0.000000 L of water were used since the beginning.


20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 395ms/step - accuracy: 0.1236 - loss: 2.3013 - val_accuracy: 0.1750 - val_loss: 2.2895
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 385ms/step - accuracy: 0.1534 - loss: 2.2456 - val_accuracy: 0.1437 - val_loss: 2.2710
Epoch 4/20
12/20 ━━━━━━━━━━━━━━━━━━━━ 2s 356ms/step - accuracy: 0.1713 - loss: 2.2534

[codecarbon INFO @ 12:15:31] Energy consumed for RAM : 0.000050 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:15:31] Delta energy consumed for CPU with constant : 0.000021 kWh, power : 5.0 W
[codecarbon INFO @ 12:15:31] Energy consumed for All CPU : 0.000042 kWh
[codecarbon INFO @ 12:15:31] 0.000092 kWh of electricity and 0.000000 L of water were used since the beginning.


...

20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 390ms/step - accuracy: 0.4147 - loss: 1.6206 - val_accuracy: 0.4187 - val_loss: 1.6627
Epoch 19/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - accuracy: 0.4845 - loss: 1.3902

[codecarbon INFO @ 12:17:31] Energy consumed for RAM : 0.000250 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:17:31] Delta energy consumed for CPU with constant : 0.000021 kWh, power : 5.0 W
[codecarbon INFO @ 12:17:31] Energy consumed for All CPU : 0.000208 kWh
[codecarbon INFO @ 12:17:31] 0.000458 kWh of electricity and 0.000000 L of water were used since the beginning.


20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 403ms/step - accuracy: 0.4930 - loss: 1.4123 - val_accuracy: 0.3750 - val_loss: 1.7858
Epoch 20/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 8s 406ms/step - accuracy: 0.5415 - loss: 1.3379 - val_accuracy: 0.4688 - val_loss: 1.5695


[codecarbon INFO @ 12:17:40] Energy consumed for RAM : 0.000264 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:17:40] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 5.0 W
[codecarbon INFO @ 12:17:40] Energy consumed for All CPU : 0.000220 kWh
[codecarbon INFO @ 12:17:40] 0.000484 kWh of electricity and 0.000000 L of water were used since the beginning.


Elapsed time (s): 158.7164590358734
Estimated CO2 emissions (kg): 8.429044951938013e-05


**Evaluation of de CNN model with the test**

In [34]:
test_loss_cnn, test_acc_cnn = cnn_model.evaluate(test_gen_cnn)
print("Test loss (CNN scratch):", test_loss_cnn)
print("Test accuracy (CNN scratch):", test_acc_cnn)

y_prob_cnn = cnn_model.predict(test_gen_cnn)
y_pred_cnn = np.argmax(y_prob_cnn, axis=1)
y_true_cnn = test_gen_cnn.classes
class_indices = test_gen_cnn.class_indices
idx_to_class = {v: k for k, v in class_indices.items()}

print(classification_report(y_true_cnn, y_pred_cnn, target_names=[idx_to_class[i] for i in range(len(idx_to_class))]))
print(confusion_matrix(y_true_cnn, y_pred_cnn))

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.5100 - loss: 1.4649 
Test loss (CNN scratch): 1.4648993015289307
Test accuracy (CNN scratch): 0.5099999904632568
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step
              precision    recall  f1-score   support

       blues       0.47      0.35      0.40        20
   classical       0.70      0.80      0.74        20
     country       0.38      0.30      0.33        20
       disco       0.26      0.40      0.31        20
      hiphop       0.52      0.65      0.58        20
        jazz       0.76      0.65      0.70        20
       metal       0.78      0.70      0.74        20
         pop       0.67      0.30      0.41        20
      reggae       0.48      0.65      0.55        20
        rock       0.32      0.30      0.31        20

    accuracy                           0.51       200
   macro avg       0.53      0.51      0.51       200
weighted avg       0.53      0.51      0.51       200

[[ 7  1  2  5  1  0  0  0  2  2]
 [ 1

## _TASK 2_

**We generate the data for the pretrained model (TASK 2)**

In [35]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

img_size_vgg = (224, 224)

train_datagen_vgg = ImageDataGenerator(preprocessing_function=preprocess_input)
val_datagen_vgg = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen_vgg = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen_vgg = train_datagen_vgg.flow_from_dataframe(
    train_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size_vgg,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True
)

val_gen_vgg = val_datagen_vgg.flow_from_dataframe(
    val_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size_vgg,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True
)

test_gen_vgg = test_datagen_vgg.flow_from_dataframe(
    test_df,
    x_col="filepath",
    y_col="label",
    target_size=img_size_vgg,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=False
)

Found 639 validated image filenames belonging to 10 classes.
Found 160 validated image filenames belonging to 10 classes.
Found 200 validated image filenames belonging to 10 classes.


**Pretrained model with VGG16 and new classification head**

In [36]:
base_model = VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(img_size_vgg[0], img_size_vgg[1], 3)
)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(num_classes, activation="softmax")(x)

vgg_model = models.Model(inputs=base_model.input, outputs=output)

vgg_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

vgg_model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,586 (56.64 MB)

 Trainable params: 133,898 (523.04 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

**Training of the models head and measurement of impact (CO2)**


In [37]:
epochs_vgg_head = 10

tracker_vgg_head = EmissionsTracker(project_name="lab2_vgg_head")
tracker_vgg_head.start()
start_time_vgg_head = time.time()

history_vgg_head = vgg_model.fit(
    train_gen_vgg,
    epochs=epochs_vgg_head,
    validation_data=val_gen_vgg
)

end_time_vgg_head = time.time()
emissions_vgg_head = tracker_vgg_head.stop()

elapsed_vgg_head = end_time_vgg_head - start_time_vgg_head
print("Elapsed time head (s):", elapsed_vgg_head)
print("Estimated CO2 emissions head (kg):", emissions_vgg_head)

[codecarbon WARNING @ 12:17:47] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 12:17:47] Error while trying to count physical CPUs: [Errno 2] No such file or directory: 'lscpu'. Defaulting to 1.
[codecarbon INFO @ 12:17:47] [setup] RAM Tracking...
[codecarbon INFO @ 12:17:47] [setup] CPU Tracking...
[codecarbon WARNING @ 12:17:47] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS and ARM processor detected: Please enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 12:17:47] CPU Model on constant consumption mode: Apple M1 Pro
[codecarbon WARNING @ 12:17:47] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 12:17:47] [setup] GPU Tracking...
[codecarbon INFO @ 12:17:47] No GPU found.
[codecarbon INFO @ 12:17:47] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global const

Epoch 1/10
 6/20 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.0791 - loss: 11.9786

[codecarbon INFO @ 12:18:03] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:18:03] Delta energy consumed for CPU with constant : 0.000021 kWh, power : 5.0 W
[codecarbon INFO @ 12:18:03] Energy consumed for All CPU : 0.000021 kWh
[codecarbon INFO @ 12:18:03] 0.000046 kWh of electricity and 0.000000 L of water were used since the beginning.


13/20 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.0913 - loss: 10.3687

[codecarbon INFO @ 12:18:18] Energy consumed for RAM : 0.000050 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:18:18] Delta energy consumed for CPU with constant : 0.000021 kWh, power : 5.0 W
[codecarbon INFO @ 12:18:18] Energy consumed for All CPU : 0.000042 kWh
[codecarbon INFO @ 12:18:18] 0.000092 kWh of electricity and 0.000000 L of water were used since the beginning.


...

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6164 - loss: 1.0721

[codecarbon INFO @ 12:26:34] Energy consumed for RAM : 0.000873 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:26:34] Delta energy consumed for CPU with constant : 0.000021 kWh, power : 5.0 W
[codecarbon INFO @ 12:26:34] Energy consumed for All CPU : 0.000729 kWh
[codecarbon INFO @ 12:26:34] 0.001602 kWh of electricity and 0.000000 L of water were used since the beginning.


20/20 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - accuracy: 0.6228 - loss: 1.0426 - val_accuracy: 0.5500 - val_loss: 1.3164


[codecarbon INFO @ 12:26:38] Energy consumed for RAM : 0.000881 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:26:38] Delta energy consumed for CPU with constant : 0.000006 kWh, power : 5.0 W
[codecarbon INFO @ 12:26:38] Energy consumed for All CPU : 0.000735 kWh
[codecarbon INFO @ 12:26:38] 0.001616 kWh of electricity and 0.000000 L of water were used since the beginning.


Elapsed time head (s): 529.8633680343628
Estimated CO2 emissions head (kg): 0.00028122554494707923


**Fine tuning of the last convolutional block**

In [40]:
for layer in base_model.layers[-4:]:
    layer.trainable = True

vgg_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

epochs_vgg_ft = 5

tracker_vgg_ft = EmissionsTracker(project_name="lab2_vgg_finetune")
tracker_vgg_ft.start()
start_time_vgg_ft = time.time()

history_vgg_ft = vgg_model.fit(
    train_gen_vgg,
    epochs=epochs_vgg_ft,
    validation_data=val_gen_vgg
)

end_time_vgg_ft = time.time()
emissions_vgg_ft = tracker_vgg_ft.stop()

elapsed_vgg_ft = end_time_vgg_ft - start_time_vgg_ft
print("Elapsed time fine-tune (s):", elapsed_vgg_ft)
print("Estimated CO2 emissions fine-tune (kg):", emissions_vgg_ft)

[codecarbon WARNING @ 12:27:20] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 12:27:20] Error while trying to count physical CPUs: [Errno 2] No such file or directory: 'lscpu'. Defaulting to 1.
[codecarbon INFO @ 12:27:20] [setup] RAM Tracking...
[codecarbon INFO @ 12:27:20] [setup] CPU Tracking...
[codecarbon WARNING @ 12:27:20] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS and ARM processor detected: Please enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 12:27:20] CPU Model on constant consumption mode: Apple M1 Pro
[codecarbon WARNING @ 12:27:20] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 12:27:20] [setup] GPU Tracking...
[codecarbon INFO @ 12:27:20] No GPU found.
[codecarbon INFO @ 12:27:20] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global const

Epoch 1/5
 5/20 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.4954 - loss: 1.4268

[codecarbon INFO @ 12:27:39] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:27:39] Delta energy consumed for CPU with constant : 0.000021 kWh, power : 5.0 W
[codecarbon INFO @ 12:27:39] Energy consumed for All CPU : 0.000021 kWh
[codecarbon INFO @ 12:27:39] 0.000046 kWh of electricity and 0.000000 L of water were used since the beginning.


11/20 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.4763 - loss: 1.5317

[codecarbon INFO @ 12:27:54] Energy consumed for RAM : 0.000050 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:27:54] Delta energy consumed for CPU with constant : 0.000021 kWh, power : 5.0 W
[codecarbon INFO @ 12:27:54] Energy consumed for All CPU : 0.000042 kWh
[codecarbon INFO @ 12:27:54] 0.000092 kWh of electricity and 0.000000 L of water were used since the beginning.


...

18/20 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.8072 - loss: 0.5306

[codecarbon INFO @ 12:32:09] Energy consumed for RAM : 0.000475 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:32:09] Delta energy consumed for CPU with constant : 0.000021 kWh, power : 5.0 W
[codecarbon INFO @ 12:32:09] Energy consumed for All CPU : 0.000396 kWh
[codecarbon INFO @ 12:32:09] 0.000871 kWh of electricity and 0.000000 L of water were used since the beginning.


20/20 ━━━━━━━━━━━━━━━━━━━━ 59s 3s/step - accuracy: 0.8372 - loss: 0.4995 - val_accuracy: 0.6250 - val_loss: 1.3519


[codecarbon INFO @ 12:32:23] Energy consumed for RAM : 0.000498 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:32:23] Delta energy consumed for CPU with constant : 0.000019 kWh, power : 5.0 W
[codecarbon INFO @ 12:32:23] Energy consumed for All CPU : 0.000415 kWh
[codecarbon INFO @ 12:32:23] 0.000913 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon WARNING @ 12:32:23] The CSV format has changed, backing up old emission file.


Elapsed time fine-tune (s): 298.93124198913574
Estimated CO2 emissions fine-tune (kg): 0.0001588450080252739


**Evaluation of the pretrained model along with the testing**

In [41]:
test_loss_vgg, test_acc_vgg = vgg_model.evaluate(test_gen_vgg)
print("Test loss (VGG):", test_loss_vgg)
print("Test accuracy (VGG):", test_acc_vgg)

y_prob_vgg = vgg_model.predict(test_gen_vgg)
y_pred_vgg = np.argmax(y_prob_vgg, axis=1)
y_true_vgg = test_gen_vgg.classes
class_indices_vgg = test_gen_vgg.class_indices
idx_to_class_vgg = {v: k for k, v in class_indices_vgg.items()}

print(classification_report(y_true_vgg, y_pred_vgg, target_names=[idx_to_class_vgg[i] for i in range(len(idx_to_class_vgg))]))
print(confusion_matrix(y_true_vgg, y_pred_vgg))

7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.6350 - loss: 1.2748
Test loss (VGG): 1.274752140045166
Test accuracy (VGG): 0.6349999904632568
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step
              precision    recall  f1-score   support

       blues       0.80      0.40      0.53        20
   classical       0.85      0.85      0.85        20
     country       0.62      0.65      0.63        20
       disco       0.65      0.65      0.65        20
      hiphop       0.51      0.90      0.65        20
        jazz       0.61      0.85      0.71        20
       metal       0.81      0.85      0.83        20
         pop       0.62      0.40      0.48        20
      reggae       0.69      0.45      0.55        20
        rock       0.37      0.35      0.36        20

    accuracy                           0.64       200
   macro avg       0.65      0.64      0.62       200
weighted avg       0.65      0.64      0.62       200

[[ 8  0  3  1  2  5  1  0  0  0]
 [ 0 17  0  0  0  1  0  0

**Loading of the features CSV for comparison**

In [42]:
features_30_path = data_dir / "lab2_features_30_sec.csv"
features_3_path = data_dir / "lab2_features_3_sec.csv"

if features_30_path.exists():
    f30 = pd.read_csv(features_30_path)
    print("lab2_features_30_sec shape:", f30.shape)

if features_3_path.exists():
    f3 = pd.read_csv(features_3_path)
    print("lab2_features_3_sec shape:", f3.shape)

lab2_features_30_sec shape: (1000, 60)
lab2_features_3_sec shape: (9990, 60)
